In [1]:
#Подключим нужные библиотеки
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from catboost import CatBoostRegressor, Pool

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from scipy.stats import randint, uniform

Так как ранее данные уже были проанализированны и очишенны от выбросов и неэффективгых признаков сразу приступаем к построению моделей.

In [2]:
#Загрузи данные
IC50_df = pd.read_csv('./data/IC50.csv')
display(IC50_df)

,Unnamed: 0,"IC50, mM",MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MaxPartialCharge,MaxAbsPartialCharge,FpDensityMorgan1,...,fr_lactone,fr_nitro,fr_piperdine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_tetrazole,fr_thiazole,fr_unbrch_alkane
0,0,6.239374,5.094096,0.387225,0.387225,0.417362,42.928571,0.038844,0.293526,0.642857,...,0,0,0,0,0,0,0,0,0,3
1,1,0.771831,3.961417,0.533868,0.533868,0.462473,45.214286,0.012887,0.313407,0.607143,...,0,0,0,0,0,0,0,0,0,3
2,2,223.808778,2.627117,0.543231,0.543231,0.260923,42.187500,0.094802,0.325573,0.562500,...,0,0,0,0,0,2,0,0,0,3
3,3,1.705624,5.097360,0.390603,0.390603,0.377846,41.862069,0.038844,0.293526,0.620690,...,0,0,0,0,0,0,0,0,0,4
4,4,107.131532,5.150510,0.270476,0.270476,0.429038,36.514286,0.062897,0.257239,0.600000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,996,31.000104,12.934891,0.048029,-0.476142,0.382752,49.133333,0.317890,0.468587,1.133333,...,2,0,0,0,0,0,0,0,0,0
886,997,31.999934,13.635345,0.030329,-0.699355,0.369425,44.542857,0.327562,0.467493,1.085714,...,2,0,0,0,0,0,0,0,0,0
887,998,30.999883,13.991690,0.026535,-0.650790,0.284923,41.973684,0.327887,0.467485,1.157895,...,2,0,0,0,0,0,1,0,0,0
888,999,31.998959,13.830180,0.146522,-1.408652,0.381559,39.000000,0.312509,0.468755,0.756757,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Выведем целевой признак
target = IC50_df['IC50, mM']
features = IC50_df.drop(['IC50, mM', 'Unnamed: 0'], axis=1)

In [4]:
#Выполним стандартизацию данных
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [5]:
#Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [100]:
#Опишем функцию печати метрик модели
def print_model_metrics(iTest, iPredict):
    # Средняя абсолютная ошибка (MAE)
    mae = mean_absolute_error(iTest, iPredict)
    print('{0:4} = {1}'.format('MAE', mae))
    
    # Среднеквадратичная ошибка (MSE)
    mse = mean_squared_error(iTest, iPredict)
    print('{0:4} = {1}'.format('MSE', mse))
    
    # Корень из среднеквадратичной ошибки (RMSE)
    rmse = np.sqrt(mse)
    print('{0:4} = {1}'.format('RMSE', rmse))
    
    # Коэффициент детерминации (R2)
    r2 = r2_score(iTest, iPredict)
    print('{0:4} = {1}'.format('R2', r2))

<h2>Модель линейной регрессии</h2>

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Линейная регрессия:')
print_model_metrics(y_test, y_pred)

Линейная регрессия:
MAE  = 210.00514263117117
MSE  = 80689.41580423445
RMSE = 284.0588245491318
R2   = 0.3833290008665948


<h2>Модель DecisionTree</h2>

In [62]:
model = DecisionTreeRegressor(
    max_depth=7,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Дерево решений:')
print_model_metrics(y_test, y_pred)

Дерево решений:
MAE  = 178.14967004552653
MSE  = 79025.28584728057
RMSE = 281.1143643560047
R2   = 0.39604715817402114


<h2>Модель Random Forest</h2>

In [99]:
#Обучение модели
model = RandomForestRegressor(
    n_estimators=35,
    max_depth=9,
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Ансамбль деревьев:')
print_model_metrics(y_test, y_pred)

Ансамбль деревьев:
MAE  = 176.23588470435337
MSE  = 68954.82002436121
RMSE = 262.5924980351899
R2   = 0.47301096016541677


<h2>Модель SVR</h2>

In [97]:
# Обучение модели
model = SVR(
    kernel='rbf',
    C=500,
    epsilon=10
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия по опорным векторам:')
print_model_metrics(y_test, y_pred)

Регрессия по опорным векторам:
MAE  = 167.58319849992634
MSE  = 85078.63942161454
RMSE = 291.6824290587531
R2   = 0.3497842430247903


<h2>Модель KNN</h2>

In [27]:
# Обучение модели
model = KNeighborsRegressor(
    n_neighbors=12,
    weights='distance'
)
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия по K-ближайшим соседям:')
print_model_metrics(y_test, y_pred)

Регрессия по K-ближайшим соседям:
MAE  = 174.26155006257667
MSE  = 74818.17836832178
RMSE = 273.52911795332096
R2   = 0.42820008860055403


In [12]:
model = CatBoostRegressor(
    iterations=100,
    learning_rate=0.4,
    depth=10,
    loss_function='RMSE',
    verbose=100,
    random_state=42
)

# Обучение модели
model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Предсказание
y_pred = model.predict(X_test)
print('Регрессия CatBoost:')
print_model_metrics(y_test, y_pred)

0:	learn: 313.6364996	test: 320.0299975	best: 320.0299975 (0)	total: 64.7ms	remaining: 6.41s
99:	learn: 103.8311995	test: 274.9940749	best: 270.2709573 (25)	total: 1.52s	remaining: 0us

bestTest = 270.2709573
bestIteration = 25

Shrink model to first 26 iterations.
Регрессия CatBoost:
MAE  = 167.7382084339612
MSE  = 73046.38983535474
RMSE = 270.27095632967064
R2   = 0.44174102942888693


<b>Для лучшей модели попробуем автоматический подбор гипер параметров</b>

In [63]:
param_dist = {
    'n_estimators': randint(3, 100),
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'criterion': ['friedman_mse', 'absolute_error', 'poisson'],
    'max_samples': uniform(0.1, 0.9),
}
random_search = RandomizedSearchCV(
    estimator= RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=100,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-10
)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print("Лучшие параметры:", best_params)

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print('Лучшие метрики Random Forest при подборе:')
print_model_metrics(y_test, y_pred)


Лучшие параметры: {'criterion': 'poisson', 'max_depth': 12, 'max_features': 'sqrt', 'max_samples': 0.6633457030914155, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 42}
Лучшие метрики Random Forest при подборе:
MAE  = 178.85579905286704
MSE  = 71324.17295300814
RMSE = 267.0658588307538
R2   = 0.4549031176033481


<h2>Выводы</h2>

Для прогнозирования показателя <b>IC50</b>, были использованы основные модели классического ML, такие как:

* Linear regression;
* DecisionTree;
* Random Forest;
* SVR;
* KNN.

А так же готовое решение CatBoostRegressor.

Наилучшую метрику R2 показала модель Random Forest со значением 0.47, что считается достаточно низким. Возможно не хватает данных для улучшения прогнозирования.

Для подбора наилучших гипер параметров был использован алгоритм случайного поиска. Но ручной подбор гипер параметров оказался лучше.